# Dreem Open Datasets validation

https://pubmed.ncbi.nlm.nih.gov/32746326/

https://github.com/Dreem-Organization/dreem-learning-open

https://github.com/Dreem-Organization/dreem-learning-evaluation

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import pingouin as pg
import sklearn.metrics as skm
from tqdm.notebook import tqdm
import scipy.stats as sp_stats
import matplotlib.pyplot as plt
sns.set(style="ticks", font_scale=1.1)

# Load predictions file
model = "eeg+eog+emg+demo"
path_dodh = "output/cv/%s/pred_dreem_dodh.csv" % model
path_dodo = "output/cv/%s/pred_dreem_dodo.csv" % model

NUM2STR = {0: "W", 1: "N1", 2: "N2", 3: "N3", 4: "R"}
STR2NUM = {"W": 0, "N1": 1, "N2": 2, "N3": 3, "R": 4}

df = pd.concat([
    pd.read_csv(path_dodh, index_col=[0, 1, 2]),
    pd.read_csv(path_dodo, index_col=[0, 1, 2])
])

# Map stages
labels = ['N1', 'N2', 'N3', 'R', 'W']
cols_stage = df.columns.tolist()[:-2]
print(cols_stage)

for c in cols_stage:
    df[c] = df[c].replace(NUM2STR)
    assert np.unique(df[c]).tolist() == labels

df.reset_index(inplace=True)

# Optional: keep specific dataset
# df = df[df['dataset'] == 'dodh'].reset_index(drop=True)

# Optional: remove subjects for which the hypnogram doesn't match the EEG size
df = df[df['pad'] <= 2].reset_index(drop=True)

print(df['subj'].nunique(), 'subjects')
print(df.shape)
df.head().round(2)

In [ ]:
# (Optional) To avoid invalid F1, we remove subjects that do not have all sleep stages in 
# original scoring. If disabled, make sure to use f1_score(zero_division=1).
# n_stage_per_subj = df.groupby('subj')['cons'].nunique()
# bad_ss = n_stage_per_subj[n_stage_per_subj != 5].index
# df = df[~df['subj'].isin(bad_ss)].reset_index(drop=True)
# print(df['subj'].nunique(), 'remaining subjects')
# print(df.shape)

### Descriptive statistics

In [ ]:
print(f"{df.shape[0] / 120:.2f} hours of data")

## Performance

In [ ]:
# Overall (not reported in the paper)
# print("Acc.:\t  %.3f" % skm.accuracy_score(df['cons'], df['yasa']))
# print("Kappa:\t  %.3f" % skm.cohen_kappa_score(df['cons'], df['yasa']))
# print("MCC:\t  %.3f" % skm.matthews_corrcoef(df['cons'], df['yasa']))
# print("F1-macro: %.3f" % skm.f1_score(df['cons'], df['yasa'], average='macro'))
# print("F1-micro: %.3f" % skm.f1_score(df['cons'], df['yasa'], average='micro'))

In [ ]:
# Per each night
df_scores = []

def perc_transition(col):
    return (col != col.shift(1)).sum() / col.shape[0]

for sub in tqdm(df['subj'].unique(), leave=False):
    df_sub = df[df['subj'] == sub]
    yt = df_sub['cons']
    yp = df_sub['yasa']
    n = yt.shape[0]

    sub_scores = {
        'dataset': df_sub['dataset'].iloc[0],
        'scorer': "yasa",
        "pad": df_sub['pad'].iloc[0],
        'dur_min': yt.size / 2,
        # % Transitions
        'perc_trans_true': perc_transition(yt),
        'perc_trans_pred': perc_transition(yp),
        # Accuracy
        'accuracy': skm.accuracy_score(yt, yp),
        'kappa': skm.cohen_kappa_score(yt, yp),
        'MCC': skm.matthews_corrcoef(yt, yp),
        'f1_macro': skm.f1_score(yt, yp, average='macro', zero_division=1),
    }

    # F1 for each stage
    f1 = skm.f1_score(yt, yp, average=None, labels=labels, zero_division=1)
    for f, l in zip(f1, labels):
        sub_scores['f1_' + l] = f

    # Proportion of each stage (NaN = 0)
    prop_true = (yt.value_counts() / n).add_prefix('perc_').add_suffix('_true')
    prop_pred = (yp.value_counts() / n).add_prefix('perc_').add_suffix('_pred')
    sub_scores.update(prop_true.to_dict())
    sub_scores.update(prop_pred.to_dict())

    # Append to main dataframe
    df_scores.append(pd.DataFrame(sub_scores, index=[sub]))

df_scores = pd.concat(df_scores)
df_scores.index.name = 'subj'
df_scores = df_scores.sort_index(axis=1).set_index("scorer", append=True)
df_scores.head().round(3)

In [ ]:
# Fill the NaN in perc_XX by zero: CAREFUL
# df_scores.isna().sum(0)
df_scores.fillna(0, inplace=True)

In [ ]:
# Show the median
df_scores.median().round(3)

### Boxplots


In [ ]:
cmap = list(sns.color_palette("Blues", n_colors=10, as_cmap=False, desat=1))
color_pred = cmap[-1]
color_ref = "tab:orange"
cmap_stages = ['#99d7f1', '#009DDC', 'xkcd:twilight blue', 'xkcd:rich purple', 'xkcd:sunflower']

In [ ]:
df_f1 = df_scores[['f1_N1', 'f1_N2', 'f1_N3', 'f1_R', 'f1_W']].copy()
df_f1.columns = df_f1.columns.str.split('_').str.get(1)

fig, ax = plt.subplots(1, 1, figsize=(4.5, 4.5), dpi=100)
sns.stripplot(data=df_f1, palette=cmap_stages, ax=ax, alpha=0.5, linewidth=0.75)

plt.xlabel("Stage")
plt.ylabel("F1-score")
plt.ylim(0, 1)
sns.despine()

### Stage discrepancies

In [ ]:
# Effect size of the percentage of transitions
def mean_std(x):
    x = x * 100
    return f"{x.mean().round(1)} ± {x.std(). round(1)}"

display(df_scores[['perc_trans_pred', 'perc_trans_true']].agg(mean_std))
pg.ttest(df_scores['perc_trans_pred'], df_scores['perc_trans_true'], paired=True).round(2)

In [ ]:
df_prop_pred = df_scores.filter(like="_pred").iloc[:, :-1].melt(var_name="Stage", value_name="Proportion", ignore_index=False)
df_prop_true = df_scores.filter(like="_true").iloc[:, :-1].melt(var_name="Stage", value_name="Proportion", ignore_index=False)

df_prop_pred['Stage'] = df_prop_pred['Stage'].str.split('_').str.get(1)
df_prop_true['Stage'] = df_prop_true['Stage'].str.split('_').str.get(1)

df_prop_pred['Scoring'] = 'Predicted'
df_prop_true['Scoring'] = 'Reference'

df_prop = pd.concat((df_prop_pred.reset_index(), df_prop_true.reset_index()))
df_prop = df_prop.sort_values(by=['subj', 'Stage', 'Scoring']).reset_index(drop=True)

df_prop

In [ ]:
# Calculate the effect size
ptest = df_prop.pairwise_ttests(dv="Proportion", within=['Stage', "Scoring"], subject="subj", effsize="cohen").iloc[11:, :].round(3)
ef = ptest.loc[:, ['Stage', 'cohen']].set_index("Stage").abs()
display(ef)

# Boxplot
fig, ax = plt.subplots(1, 1, figsize=(4.5, 4.5), dpi=100)

sns.boxplot(y=df_prop['Proportion'] * 100, x=df_prop['Stage'], hue=df_prop['Scoring'],
            hue_order=['Reference', 'Predicted'], 
            palette=[color_ref, color_pred], 
            saturation=1, width=0.6, fliersize=0, linewidth=1.5, notch=True)

plt.ylim(0, 80)
plt.yticks([0, 20, 40, 60, 80])
plt.legend(frameon=False, loc="upper right")
plt.ylabel("Proportion of time in bed (%)");

sns.despine()
plt.tight_layout()

#### Confusion matrices

The normalized confusion matrices show the sensitivity (= recall).

In [ ]:
cm = 100 * skm.confusion_matrix(df['cons'], df['yasa'], labels=labels, normalize="true")
cm = pd.DataFrame(cm, index=labels, columns=labels).round(1)

# Plot
fig, ax = plt.subplots(1, 1, dpi=100, figsize=(4.5, 4.5))
sns.heatmap(cm, annot=True, vmin=0, vmax=100, cmap="Blues", square=True, cbar=False, fmt=".1f")
plt.ylabel("Reference")
plt.xlabel("Predicted")
plt.tight_layout()

In [ ]:
# # Precision 
# # skm.precision_recall_fscore_support(df['y_true'], df['y_pred'], labels=labels)
# cm = 100 * skm.confusion_matrix(df['cons'], df['yasa'], labels=labels, normalize="pred")
# cm = pd.DataFrame(cm, index=labels, columns=labels).round(1)

# # Plot
# fig, ax = plt.subplots(1, 1, dpi=100, figsize=(4.5, 4.5))
# sns.heatmap(cm, annot=True, vmin=0, vmax=100, cmap="Blues", square=True, cbar=False, fmt=".1f")
# plt.ylabel("Reference")
# plt.xlabel("Predicted")
# plt.tight_layout()

### Stage transitions

Are most of the errors located around transitions between stages?

Here, we use PSG to define the transitions between stages.

In [ ]:
df_trans = []

for sub in tqdm(df['subj'].unique(), leave=False):
    df_sub = df[df['subj'] == sub]
    yt = df_sub['cons']
    yp = df_sub['yasa']

    # Identify stable periods, i.e. the 3 epochs before / after are similar (3 minutes window)
    first_ep, last_ep = yt.iloc[0], yt.iloc[-1]
    stable = np.logical_and.reduce((
        yt.shift(1, fill_value=first_ep) == yt,  # = same as previous one
        yt.shift(-1, fill_value=last_ep) == yt, # = same as next one
        yt.shift(2, fill_value=first_ep) == yt,
        yt.shift(-2, fill_value=last_ep) == yt,
        yt.shift(3, fill_value=first_ep) == yt,
        yt.shift(-3, fill_value=last_ep) == yt,
    ))

    # Append to main dict
    sub_scores = {
        'n_stable': len(stable[stable]),
        'n_trans': len(stable[~stable]),
        'acc_stable': skm.accuracy_score(yt[stable], yp[stable]),
        'acc_trans': skm.accuracy_score(yt[~stable], yp[~stable]),
        'mcc_stable': skm.matthews_corrcoef(yt[stable], yp[stable]),
        'mcc_trans': skm.matthews_corrcoef(yt[~stable], yp[~stable])
    }

    # Append to main dataframe
    df_trans.append(pd.DataFrame(sub_scores, index=[sub]))

df_trans = pd.concat(df_trans)
df_trans.sort_index(axis=1, inplace=True)
df_trans.index.name = 'subj'
df_trans.round(3)

In [ ]:
# Average and T-test
display(df_trans[['acc_stable', 'acc_trans']].apply(mean_std))

pg.ttest(df_trans['acc_stable'], df_trans['acc_trans'], paired=False)
# pg.ttest(df_trans['mcc_stable'], df_trans['mcc_trans'], paired=False)

In [ ]:
# Plot
sns.set(style="ticks", font_scale=1.1)
fig, ax = plt.subplots(1, 1, figsize=(2.5, 3.5), dpi=100)
sns.boxplot(data=df_trans[['acc_stable', 'acc_trans']], width=0.7, fliersize=0,
            saturation=1, color=color_pred, ax=ax)

plt.ylim(0.5, 1.01)
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.xticks([0, 1], ['Stable', 'Transition'])
sns.despine()
plt.tight_layout()

### High vs low confidence

In [ ]:
df_conf = []

for sub in tqdm(df['subj'].unique(), leave=False):
    df_sub = df[df['subj'] == sub]
    yt = df_sub['cons']
    yp = df_sub['yasa']
    
    highconf = df_sub['confidence'] >= 0.8

    # Append to main dict
    sub_scores = {
        'n_highconf': len(highconf[highconf]),
        'n_lowconf': len(highconf[~highconf]),
        'acc_highconf': skm.accuracy_score(yt[highconf], yp[highconf]),
        'acc_lowconf': skm.accuracy_score(yt[~highconf], yp[~highconf]),
        'mcc_highconf': skm.matthews_corrcoef(yt[highconf], yp[highconf]),
        'mcc_lowconf': skm.matthews_corrcoef(yt[~highconf], yp[~highconf])
    }

    # Append to main dataframe
    df_conf.append(pd.DataFrame(sub_scores, index=[sub]))

df_conf = pd.concat(df_conf)
df_conf.sort_index(axis=1, inplace=True)
df_conf.index.name = 'subj'
df_conf.round(3)

In [ ]:
display(df_conf[['acc_highconf', 'acc_lowconf']].apply(mean_std))
pg.ttest(df_conf['acc_highconf'], df_conf['acc_lowconf'], paired=False)

********

## How does YASA perform compared to the other human scorers?

In [ ]:
def consensus_score(data):
    """Create consensus score on N-1 scorers (= 4 scorers).
    
    When ties are present (e.g. [0, 0, 1, 1]), use the scoring of the
    most reliable scorer of the record, i.e. the one with the overall strongest
    agreement with all the other ones.
    """
    # Reset index so that .loc = .iloc
    data = data.reset_index(drop=True)
    # Calculate pairwise agreement between scorer
    corr_acc = data.replace(STR2NUM).corr(skm.accuracy_score).mean()
    # Find index of best scorer
    idx_best_scorer = corr_acc.sort_values(ascending=False).index[0]
    # Calculate consensus stage
    mode, counts = sp_stats.mode(data, axis=1)
    mode = np.squeeze(mode)
    counts = np.squeeze(counts)
    # Find indices of ties
    ties = np.where(counts == 2)[0]
    # Replace ties values by most reliable scorer of the record
    mode[ties] = data.loc[ties, idx_best_scorer].to_numpy()
    return mode

In [ ]:
cols_scorer = df.columns[df.columns.str.startswith("scorer")].tolist()

df_scores_human = []

# Loop across nights
for sub in tqdm(df['subj'].unique(), leave=False):
    df_sub = df[df['subj'] == sub]
    for c in cols_scorer:
        # Consensus excluding current scorer
        other_scorers = np.setdiff1d(cols_scorer, c).tolist()
        yt = consensus_score(df_sub[other_scorers])
        yp = df_sub[c]
        # Calculate performance metrics
        sub_scores = {
            'subj': sub,
            'scorer': c,
            'dataset': df_sub['dataset'].iloc[0],
            'accuracy': skm.accuracy_score(yt, yp),
            'kappa': skm.cohen_kappa_score(yt, yp),
            'MCC': skm.matthews_corrcoef(yt, yp),
            'f1_macro': skm.f1_score(yt, yp, average='macro', zero_division=1),
        }

        # F1 for each stage
        f1 = skm.f1_score(yt, yp, average=None, labels=labels, zero_division=1)
        for f, l in zip(f1, labels):
            sub_scores['f1_' + l] = f
        
        df_scores_human.append(pd.DataFrame(sub_scores, index=[0]))
        
df_scores_human = pd.concat(df_scores_human, ignore_index=True).set_index(["subj", "scorer"])
# Append YASA scoring
df_scores = pd.concat([df_scores, df_scores_human], axis=0, join='inner').sort_index()
df_scores.round(3)

In [ ]:
# Mean + STD
df_scores.groupby("scorer").agg(mean_std)

In [ ]:
# Ranks are calculated separately for each night and then averaged
# df_scores.groupby("subj").rank(ascending=False).groupby("scorer").mean().round(2)

In [ ]:
# Pairwise T-test of YASA against the other scorers
(df_scores
 .reset_index()
 .pairwise_ttests(dv="accuracy", within="scorer", subject="subj", return_desc=True)
 .drop(columns=['Contrast', 'Paired', "Parametric", "BF10", "Tail", "hedges"])
 .set_index(['A', 'B'])
 .xs("yasa", level=1, drop_level=False)
 .round(3)
)

In [ ]:
# Same for cohen kappa
(df_scores
 .reset_index()
 .pairwise_ttests(dv="kappa", within="scorer", subject="subj", return_desc=True)
 .drop(columns=['Contrast', 'Paired', "Parametric", "BF10", "Tail", "hedges"])
 .set_index(['A', 'B'])
 .xs("yasa", level=1, drop_level=False)
 .round(3)
)